# Goal of notebook:


# Load modules 

In [2]:
import scanpy as sc
import scvi
import anndata as an
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import cell2location as c2l
import scvi

from matplotlib import rcParams

import scanpy as sc
import muon as mu
import os 

/projects/home/plopez29/cell2location/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Objs
using V13 - latest version by Rachelly

In [3]:
sc_h5mu_obj = mu.read("/projects/Pregnancy_placenta/Results/10_Merging_all_batches/Batches1-15_V13.h5mu")

In [4]:
sc_h5mu_obj["GEX"].obs["Lineage"].value_counts()

Lineage
HBC              339825
Trophoblasts     320202
Myeloid          173670
fPerivascular    101267
fFibroblasts      84604
T                 68146
NK                49525
EVT               32808
dStromal          23706
Endothelial       20463
B_Plasma          13330
HSC                 889
Other               326
Name: count, dtype: int64

In [5]:
sc_obj = sc_h5mu_obj["GEX"]

In [6]:
# Step 1: Transfer .raw to .X (this is fast)
sc_obj.X = sc_obj.raw.X
sc_obj.raw = None  # Optional: clears memory


## Load TMA obj 

In [7]:
os.chdir("/projects/Pregnancy_placenta/Results/")
TMA_obj = an.read_h5ad("14_visium_deconvolution/visium_data/TMA_merged_b2c_V1.h5ad")

In [ ]:
TMA_obj.var["feature_name"] = TMA_obj.var_names
TMA_obj.var.set_index("gene_ids", drop=True, inplace=True)

In [ ]:
# find mitochondrial (MT) genes
TMA_obj.var["MT_gene"] = [
    gene.startswith("MT-") for gene in TMA_obj.var["feature_name"]
]

# remove MT genes for spatial mapping (keeping their counts in the object)
TMA_obj.obsm["MT"] = TMA_obj[:, TMA_obj.var["MT_gene"].values].X.toarray()
TMA_obj = TMA_obj[:, ~TMA_obj.var["MT_gene"].values]

In [ ]:
TMA_obj.write_h5ad("14_visium_deconvolution/visium_data/TMA_merged_b2c_V2.h5ad")

In [8]:
TMA_obj = an.read_h5ad("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/visium_data/TMA_merged_b2c_V2.h5ad")

In [12]:
TMA_obj

AnnData object with n_obs × n_vars = 317507 × 18074
    obs: 'in_tissue', 'array_row', 'array_col', 'Row_num', 'Tissue_core', 'Patient', 'Condition', 'TMA_num', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'library_id'
    var: 'feature_name', 'MT_gene'
    uns: 'spatial'
    obsm: 'MT', 'spatial'

# Prep objs

## prep sc obj 

In [10]:
# Step 1: Remove 'G:' from var_names
sc_obj.var_names = sc_obj.var_names.str.replace('^G:', '', regex=True)

# Step 2: Create a mapping from TMA_obj (assuming gene symbols are unique)
# 'feature_name' column holds gene symbols, index is Ensembl ID
gene_map = TMA_obj.var.reset_index().set_index('feature_name')

# Step 3: Map gene symbols to Ensembl IDs
sc_obj.var['feature_name'] = sc_obj.var_names  # preserve gene symbol
sc_obj.var['gene_ids'] = sc_obj.var_names.map(gene_map['gene_ids'])

# Step 4: Drop entries where no match is found (optional)
sc_obj = sc_obj[:, sc_obj.var['gene_ids'].notna()].copy()

# Step 5: Set Ensembl ID as index and drop the old index
sc_obj.var.set_index('gene_ids', drop=True, inplace=True)

# Result: sc_obj.var has Ensembl IDs as index and gene symbols as 'feature_name'


In [11]:
sc_obj

AnnData object with n_obs × n_vars = 1228761 × 17872
    obs: 'Sample', 'Batch', 'n_genes', 'n_counts', 'percent_mito', 'Lineage', 'sub-lineage2', 'leiden_res_0.5', 'Condition', 'Fetal_sex', 'Maternal_age', 'Labor', 'GA_delivery', 'DeliveryMode', 'Fetal_growth_restriction', 'Birthweight_percentile', 'Hypertensive_disorder', 'PrePreg_BMI', 'Cytowash', 'Over3h_cytowah', 'status', 'assignment', 'FetMat_Annotation', 'sub_lin', 'sub_lin2'
    var: 'n_cells', 'percent_cells', 'robust', 'highly_variable_features', 'mean', 'var', 'hvf_loess', 'hvf_rank', 'feature_name'
    uns: 'Lineage_colors', 'PCs', '_attr2type', 'genome', 'leiden_resolution', 'modality', 'pca', 'pca_features', 'stdzn_max_value', 'stdzn_mean', 'stdzn_std', 'uid'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'pca_harmony_knn_distances', 'pca_harmony_knn_indices'
    varm: 'de_res_0.5'
    layers: 'norm_counts'
    obsp: 'W_pca', 'W_pca_harmony'

## Finding shared genes

In [13]:
shared_features = [
    feature for feature in TMA_obj.var_names if feature in sc_obj.var_names
]

In [92]:
(shared_features)


['ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000187583',
 'ENSG00000187642',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000188157',
 'ENSG00000237330',
 'ENSG00000131591',
 'ENSG00000162571',
 'ENSG00000186891',
 'ENSG00000186827',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000184163',
 'ENSG00000160087',
 'ENSG00000162572',
 'ENSG00000131584',
 'ENSG00000169972',
 'ENSG00000127054',
 'ENSG00000224051',
 'ENSG00000169962',
 'ENSG00000107404',
 'ENSG00000162576',
 'ENSG00000175756',
 'ENSG00000221978',
 'ENSG00000235098',
 'ENSG00000205116',
 'ENSG00000179403',
 'ENSG00000215915',
 'ENSG00000160072',
 'ENSG00000197785',
 'ENSG00000205090',
 'ENSG00000160075',
 'ENSG00000228594',
 'ENSG00000197530',
 'ENSG00000189409',
 'ENSG00000248333',
 'ENSG00000008128',
 'ENSG00000008130',
 'ENSG00000078369',
 'ENSG00000169885',
 'ENSG00000178821',
 'ENSG00000142609',
 'ENSG00000187730',
 'ENSG00000067606',
 'ENSG00000162585',
 'ENSG00000157933',
 'ENSG00000157916',


## subset based on shared genes 

In [21]:
sc_obj = sc_obj[:, shared_features].copy()
TMA_obj = TMA_obj[:, shared_features].copy()

KeyboardInterrupt: 

## read preselected genes 
starting with test1,

In [72]:
preselected_genes = pd.read_csv("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/0_infered_cell_abundance_scRNA/Test1_AUROC_plus_extra/TEST1_AUROC_0.7_manually_added_V4.csv")

### test whether genes are in scRNA obj

In [73]:
df_shared_genes = sc_obj.var

In [74]:
df_shared_genes['is_in_unique'] = df_shared_genes['feature_name'].isin(unique_genes)
df_shared_genes

,n_cells,percent_cells,robust,highly_variable_features,mean,var,hvf_loess,hvf_rank,feature_name,is_in_unique
gene_ids,,,,,,,,,,
ENSG00000187634,5349,0.403531,True,False,0.016040,0.074513,0.070807,8718,SAMD11,False
ENSG00000188976,205862,15.530308,True,False,0.595617,2.254235,2.357804,33565,NOC2L,False
ENSG00000187961,25223,1.902833,True,False,0.069952,0.299065,0.307385,24253,KLHL17,False
ENSG00000187583,14743,1.112218,True,False,0.038770,0.156908,0.171370,32855,PLEKHN1,False
ENSG00000187642,982,0.074082,True,False,0.002692,0.011662,0.012040,20963,PERM1,False
...,...,...,...,...,...,...,...,...,...,...
ENSG00000165246,2665,0.201049,True,False,0.007005,0.029570,0.031386,27063,NLGN4Y,False
ENSG00000286265,266,0.020067,False,False,0.000719,0.003049,0.000000,-1,AC007244.1,False
ENSG00000012817,56305,4.247671,True,False,0.167231,0.727403,0.714258,9583,KDM5D,False


In [75]:
preselected_genes['is_in_sc'] = preselected_genes['gene'].isin(df_shared_genes['feature_name'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = preselected_genes.copy()

# If is_in_sc is the strings "True"/"False", coerce to boolean
if df["is_in_sc"].dtype == object:
    df["is_in_sc"] = df["is_in_sc"].map({"True": True, "False": False})

# Count genes per cell_type / clust_from / is_in_sc
count_df = (
    df.groupby(['clust_from', 'cell_type', 'is_in_sc'])
      .size()
      .reset_index(name='count')
)

# Proportion within each (clust_from, cell_type)
count_df['prop'] = (
    count_df.groupby(['clust_from', 'cell_type'])['count']
            .transform(lambda s: s / s.sum())   # <-- fixes your ValueError
)

# Faceted stacked bars: one figure per clust_from
for c, sub in count_df.groupby('clust_from'):
    wide = (sub.pivot(index='cell_type', columns='is_in_sc', values='prop')
               .fillna(0)
               .sort_index())  # optional sort

    ax = wide.plot(kind='bar', stacked=True, figsize=(10, 4))
    ax.set_title(str(c))
    ax.set_ylabel('Proportion of genes')
    ax.set_xlabel('Cell Type')
    ax.legend(title='is_in_sc', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


### saving genes that are missing 

In [78]:
missing_genes = preselected_genes[~preselected_genes['is_in_sc']]
missing_genes.to_csv("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/0_infered_cell_abundance_scRNA/Test1_AUROC_plus_extra/missing_genes_Test1.csv")

In [ ]:
### savign genes that are used in visium

In [81]:
good_genes = preselected_genes[preselected_genes['is_in_sc']]
good_genes.to_csv("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/0_infered_cell_abundance_scRNA/Test1_AUROC_plus_extra/genes_used_in_Test1.csv")

In [83]:
len(good_genes['gene'].unique())

3243

## filt objects based on genes 

In [84]:
filt_presel_genes = df_shared_genes[df_shared_genes['is_in_unique']]

In [103]:
# Make sure both are string type to avoid mismatches
genes_to_keep = filt_presel_genes.index.astype(str)
adata_genes = sc_obj.var.index.astype(str)

# Intersection
common_genes = adata_genes.intersection(genes_to_keep)

print(f"Genes in filt_presel_genes: {len(genes_to_keep)}")
print(f"Genes in sc_obj: {len(adata_genes)}")
print(f"Overlap: {len(common_genes)}")


Genes in filt_presel_genes: 3243
Genes in sc_obj: 17872
Overlap: 3243


In [104]:

# Subset AnnData
sc_obj_sub = sc_obj[:, common_genes].copy()
TMA_obj_sub = TMA_obj[:, common_genes].copy()

# Check result
#sc_obj_sub


In [105]:
TMA_obj_sub

AnnData object with n_obs × n_vars = 317507 × 3243
    obs: 'in_tissue', 'array_row', 'array_col', 'Row_num', 'Tissue_core', 'Patient', 'Condition', 'TMA_num', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'n_counts', 'library_id'
    var: 'feature_name', 'MT_gene'
    uns: 'spatial'
    obsm: 'MT', 'spatial'

## save obj 

In [106]:
os.chdir("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/ref_data")
sc_obj_sub.write_h5ad("Batches1-15_V13_TEST1_V1.h5ad")#%%
# subset to just a few cells

In [107]:
os.chdir("/projects/Pregnancy_placenta/Results/14_visium_deconvolution/visium_data")
TMA_obj_sub.write_h5ad("TMA_merged_b2c_test1_V4.h5ad")